<a href="https://colab.research.google.com/github/Sourasky-DHLAB/Whisper/blob/main/Colab/Whisper_Speaker_Diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="direction:rtl" dir="rtl">
    <img src="http://drive.google.com/uc?export=view&id=1FnLVIqEV1Tt5rCEOIk5OxihO6xdZgTMe"><br>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">    נכתב על ידי <a href="mailto:odedzarchia@tauex.tau.ac.il">עודד זרחיה</a> | <a href="https://github.com/Sourasky-DHLAB">Github</a><br>
    <a href="https://cenlib.tau.ac.il/">הספרייה המרכזית ע"ש סוראסקי</a> | <a href="https://tau.ac.il/">אוניברסיטת תל אביב</a>
</div></p>

<div dir="rtl">
<h1><strong>1. תמלול אודיו וזיהוי דוברים עם Whisper ו-pyannote.audio</strong></h1>
<p>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
<a href="https://openai.com/blog/whisper/"> Whisper </a>הוא מודל לזיהוי דיבור מבית <a href="https://openai.com/">OpenAI</a> הזמין לציבור הרחב בקוד פתוח. מודל זה אומן על יותר מ-680 אלף שעות של שיחות באנגלית ובשפות רבות אחרות – בהן גם עברית וערבית.<br>
<a href = "https://github.com/pyannote/pyannote-audio">pyannote.audio</a> היא ספריית כלים בפייתון לזיהוי דוברים המבוססת על למידת מכונה. <br>
מחברת זו תדריך אתכם בתמלול ראיונות מוקלטים באמצעות Whisper ו-pyannote.audio. תוכלו להשתמש במחברת כפי שהיא כדי לאחסן את קבצי התמליל ב-Google Drive.<br>
לתשומת לבכם - השימוש במחברת זו ובמודלים השונים חופשי לחלוטין וללא שום עלות. בנוסף, אין הגבלה על אורך קטעי הוידאו/אודיו שניתן לתמלל באמצעות Whisper.
</p>
</div>

<h1><strong><div style="direction:rtl" dir="rtl">2. בדוק את סוג המעבד הגרפי (GPU) </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">סוג <a href="https://https://he.wikipedia.org/wiki/%D7%9E%D7%A2%D7%91%D7%93_%D7%92%D7%A8%D7%A4%D7%99">המעבד הגרפי</a> <strong>(GPU - Graphics Processing Unit)</strong> שאתם מקבלים ב-Google Colab מגדיר את המהירות שבה קטעי האודיו יתומללו. ככל שמספר <a href="https://https://he.wikipedia.org/wiki/FLOPS">הפלופס</a> <strong>(FLOPS - Floating Point Operations Per Second, פעולות נקודות צפות לשנייה)</strong> גבוה יותר, כך התמלול מהיר יותר. יחד עם זאת, גם המעבד הגרפי החלש ביותר ב-Colab מסוגל להריץ כל מודל של Whisper. יש לוודא כי בחרתם ב-GPU כמאיץ חומרה עבור מחברת זו (Runtime → Change runtime type → Hardware accelerator).</p>
</div>

<table>
  <thead>
    <tr>
      <th>GPU</th>
      <th>GPU RAM</th>
      <th>FP32 teraFLOPS</th>
      <th>Availability</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>T4</td>
      <td>16 GB</td>
      <td>8.1</td>
      <td>Free</td>
    </tr>
    <tr>
      <td>P100</td>
      <td>16 GB</td>
      <td>10.6</td>
      <td>Colab Pro</td>
    </tr>
    <tr>
      <td>V100</td>
      <td>16 GB</td>
      <td>15.7</td>
      <td>Colab Pro (Rare)</td>
    </tr>
  </tbody>
</table>


<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי לוודא את סוג המעבד הגרפי שהוקצה עבור מחברת זו. אפסו את ה-runtime של המחברת אם ברצונכם לקבל מעבד גרפי אחר (Runtime → Restart runtime).</p>

In [ ]:
# check gpu type
!nvidia-smi -L
!nvidia-smi

<div style="direction:rtl" dir="rtl"><h1><strong>3. התקנת ספריות - למידת מכונה</strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להוריד ולהתקין את ספריות לימוד המכונה הנחוצות לזיהוי הדוברים.

In [ ]:
!pip install torch==2.0.0
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.1 fastai==2.7.11
!pip install tokenizers
!pip install torchdata==0.5.1

<div style="direction:rtl" dir="rtl"><h1><strong>4. התקנת ספריות - תמלול וזיהוי דוברים</strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להוריד ולהתקין את ספריית <a href="https://github.com/openai/whisper">Whisper</a> הנחוצה לפעולת התמלול ואת ספריית <a href = https://github.com/pyannote/pyannote-audio>pyannote.audio</a> לזיהוי דוברים.</p></div>

In [ ]:
# install libraries
!pip install git+https://github.com/openai/whisper.git 
!pip install -q git+https://github.com/pyannote/pyannote-audio

<div style="direction:rtl" dir="rtl">
  <h1><strong>5. ייבוא ספריות </strong></h1>
  <div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא  כדי לייבא את הספריות הנדרשות עבור פעולות התמלול וזיהוי הדוברים.</p>
</div>

In [ ]:
# import libraries
import whisper
import os
from os.path import join
import time
import datetime
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from os.path import join

<h1><strong><div style="direction:rtl" dir="rtl">6. חיבור ל-Google Drive </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להתחבר ל-Google Drive האישי שלכם. כדי לראות את הקבצים שלכם פתחו את סייר הקבצים בתפריט השמאלי.</p></div>

In [6]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

Mounted at /content/drive


<h1 dir="rtl"><strong>7. הגדרת תיקיות </strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי להגדיר את מיקום קובץ הקלט (אודיו) וקובץ הפלט (טקסט):<br>
    <ol style="float:right;">
          <li>קובץ אודיו - מיקום קובץ האודיו לתמלול מוגדר תחת משתנה  <code>audio_file</code>.העתיקו את מיקום קובץ האודיו שברצונכם לתמלל והדביקו אותו במשתנה זה. שימרו על המרכאות.<b> יושם לב כי על קובץ הראיון להיות בפורמט<code>WAV</code></b>.<br> במידה והקובץ שברשותכם נמצא בפורמט אחר (אודיו/וידאו), ניתן להמירו באמצעות תוכנות לעריכת וידאו/אודיו המגיעות עם מערכת ההפעלה (Clipchamp, Sound Converter וכולי) או דרך אפליקציות מקוונות ברשת האינטרנט. העתיקו את</li>
      <li>קובץ תמליל - מחברת זו מניחה כי קובץ הטקסט של התמלול יישמר ב-Google Drive תחת תיקיית <code>Whisper/Transcriptions/</code>. ניתן לשנות הגדרה זו במשתנה <code>transcription_folder_path</code>. במידה ואינה קיימת, תיקייה זו תיווצר באופן אוטומטי לאחר הרצת התא.</li>
    </ol>
  </p>
</div>

In [9]:
# Set audio file location
audio_file = "/content/drive/MyDrive/Whisper/Audio/Piers_Morgan_vs_Benjamin_Netanyahu.wav"

# Assuming the text files will be placed in a folder called "Transcriptions" under "Whisper"
transcription_folder_path = "/content/drive/MyDrive/Whisper/Transcriptions/"

# Create "Transcriptions" folder if does not exist
if not os.path.exists(transcription_folder_path):
  os.makedirs(transcription_folder_path)

<h1 id="-div-style-direction-rtl-dir-rtl-8-div-"><strong><div style="direction:rtl" dir="rtl">8. טעינת מודל התמלול </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי לטעון את מודל התמלול המתאים. יושם לב כי אנו משתמשים במודל הגדול (<strong>large</strong>), המדויק ביותר עבור השפה העברית. <br> ישנם חמישה מודלים, ארבעה עם גרסאות באנגלית בלבד, המציעים פשרות שונות בין מהירות לדיוק. יושם לב כי פעולת התמלול צורכת משאבי מחשוב מרובים. לפיכך, אם ברצונכם להשתמש במודל הגדול עבור קבצים מרובים, מומלץ להריץ את המחברת בסביבה עם משאבי מחשוב מובטחים כדוגמת <a href="https://colab.research.google.com/signup">Google Colab Pro</a>. עבור תמלול באנגלית בלבד, מודלי ה-en. נוטים לתפקד טוב יותר. <br>בטבלה שלהלן מוצגים שמות המודלים הזמינים יחד עם דרישות הזיכרון והמהירות היחסית שלהם:
</div>

<table>
<thead>
<tr>
<th style="text-align:center">Size</th>
<th style="text-align:center">Parameters</th>
<th style="text-align:center">English-only model</th>
<th style="text-align:center">Multilingual model</th>
<th style="text-align:center">Required VRAM</th>
<th style="text-align:center">Relative speed</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center">tiny</td>
<td style="text-align:center">39 M</td>
<td style="text-align:center"><code>tiny.en</code></td>
<td style="text-align:center"><code>tiny</code></td>
<td style="text-align:center">~1 GB</td>
<td style="text-align:center">~32x</td>
</tr>
<tr>
<td style="text-align:center">base</td>
<td style="text-align:center">74 M</td>
<td style="text-align:center"><code>base.en</code></td>
<td style="text-align:center"><code>base</code></td>
<td style="text-align:center">~1 GB</td>
<td style="text-align:center">~16x</td>
</tr>
<tr>
<td style="text-align:center">small</td>
<td style="text-align:center">244 M</td>
<td style="text-align:center"><code>small.en</code></td>
<td style="text-align:center"><code>small</code></td>
<td style="text-align:center">~2 GB</td>
<td style="text-align:center">~6x</td>
</tr>
<tr>
<td style="text-align:center">medium</td>
<td style="text-align:center">769 M</td>
<td style="text-align:center"><code>medium.en</code></td>
<td style="text-align:center"><code>medium</code></td>
<td style="text-align:center">~5 GB</td>
<td style="text-align:center">~2x</td>
</tr>
<tr>
<td style="text-align:center">large</td>
<td style="text-align:center">2870 M</td>
<td style="text-align:center">N/A</td>
<td style="text-align:center"><code>large</code></td>
<td style="text-align:center">~10 GB</td>
<td style="text-align:center">1x</td>
</tr>
</tbody>
</table>
<p><div style="direction:rtl" dir="rtl"><br></p>

In [ ]:
# load whisper model
model = whisper.load_model("tiny.en")
print(f"\033[1m Model loaded successfully")

<h1 id="-div-style-direction-rtl-dir-rtl-9-div-"><strong><div style="direction:rtl" dir="rtl">9. קביעת שפה ומספר דוברים</div></strong></h1>
<div style="direction:rtl" dir="rtl">
 <p style="text-align: right; direction: rtl; float: right;">הריצו את התא הבא כדי לקבוע שני משתנים:
<ol style="float:right;">
  <li><code>lang</code> - הגדרת שפת התמלול. <code>he</code> משמעו עברית. ניתן להדפיס את רשימת השפות הנתמכת על ידי הרצת הפקודה <code>print(whisper.tokenizer.LANGUAGES)</code> בתא נפרד.</li>
  <li><code>num_speakers</code> - שנו בהתאם למספר הדוברים בקובץ האודיו (לדוגמה, שני מראיינים ומרואיין אחד = 3).</p>
  </li>
</ol>
</div>

In [11]:
lang = 'en'
num_speakers = 2

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">10. תמלול </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי לתמלל את קבצי ה-<code>WAV</code> בתיקיית <code>Whisper/Audio</code>. פלט התמליל, המוגדר תחת משתנה <code>segments</code>, יתקבל בחלוקה לסגמנטים (רצף של קטעי שמע בדידים). </p>

In [ ]:
# set timer
import time
start_time = time.time()
# transcribe interview file
result = model.transcribe(audio_file, verbose = False, language = lang) # to translate add task = 'transalte'
segments = result["segments"]
print(f"\033[1m--- Transcribed file in %s seconds ---" % (time.time() - start_time))

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">11. טעינת קובץ WAV </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי לטעון את קובץ ה-<code>WAV</code> ולחלץ את נתוני המטא-דאטה.</p>

In [ ]:
with contextlib.closing(wave.open(audio_file,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)
  
print(f"\033[1m--- The file contains {frames} audio frames ---")
print(f"\033[1m--- The sampling frequency is {rate} Hz ---")
print(f"\033[1m--- The duration is {duration} seconds ---")

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">12. חלוקת קובץ ה-WAV למקטעים ויצירת embedding </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי להגדיר פונקציה בשם <code>segment_embedding</code>. פונקציה זו חותכת את קובץ האודיו למקטעים על סמך נתוני הסגמנטציה שהתקבלו במהלך פעולת התמלול. לאחר מכן, הפונקציה יוצרת <a href = "https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture#:~:text=An%20embedding%20is%20a%20relatively,like%20sparse%20vectors%20representing%20words.">embedding</a> עבור כל סגמנט אודיו.</p> 

In [14]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(audio_file, clip)
  return embedding_model(waveform[None])

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">13. יצירת מערך ואחסון ה-embeddings </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי ליצור מערך <a href = "https://pypi.org/project/numpy/">numpy</a> בו יאוחסנו ה-embeddings עבור מקטעי האודיו.</p>

In [15]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">14. זיהוי דוברים </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי לזהות את הדוברים במקטעי האודיו. תא זה משתמש באלגוריתם המכונה <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html">Agglomerative Clustering</a> כדי לקבץ את ה-embeddings של מקטעי האודיו לאשכולות של מספר הדוברים שהגדרנו במשתנה <code>num_speakers</code>.</p>

In [16]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

<h1 id="-div-style-direction-rtl-dir-rtl-10-div-"><strong><div style="direction:rtl" dir="rtl">15. שמירת התמלול בקובץ טקסט </div></strong></h1>
<div style="direction:rtl" dir="rtl">
  <p style="text-align: right; direction: rtl; float: right;">
הריצו את התא הבא כדי לכתוב את תוצאות זיהוי הדוברים יחד עם התמליל לקובץ טקסט. הקובץ ישמר בתיקיית <code>Whisper/Transcriptions/</code> באותו שם כמו קובץ האודיו אך עם סיומת <code>txt</code>.</p>

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

# Extract the name of the audio file without the extension
audio_filename = os.path.splitext(os.path.basename(audio_file))[0]

# Construct the path to the transcription file
transcription_file_path = os.path.join(transcription_folder_path, audio_filename + ".txt")

f = open(transcription_file_path, "w", encoding= 'UTF-8')

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()
print(f"\033[1m--- Finished writing to {transcription_file_path} ---")